## Import Libraries

In [1]:
from os import path
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## User variables

In [2]:
# PATH TO DATA
st = study('Olfacto')
path_pow = path.join(st.path, 'feature/0_Power_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/0_Classif_Power_Encoding_EpiPerf_LowHigh_None/MF_elecs/')

# POWER & STATS PARAMETERS
freqs = [2,3,4,5,6]
nperm = 200

-> Olfacto loaded


## Power Decoding - Good Bad Odors Encoding

In [3]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

phases, bsl, subjects = ['odor'],['None'],['CHAF','VACJ','SEMC','FERJ','MICP','PIRJ','LEFC']
conds = ['low','high']
for b, su, phase in product(bsl,subjects,phases):
    #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
    bad_data = np.load(path.join(path_pow, su+'_'+phase+'_'+conds[0]+'_bipo_sel_physFT_'+b+'_pow.npz'))['xpow']
    good_data = np.load(path.join(path_pow, su+'_'+phase+'_'+conds[1]+'_bipo_sel_physFT_'+b+'_pow.npz'))['xpow']
    names = np.load(path.join(path_pow, su+'_'+phase+'_'+conds[0]+'_bipo_sel_physFT_'+b+'_pow.npz'))['labels']
    channels = np.load(path.join(path_pow, su+'_'+phase+'_'+conds[0]+'_bipo_sel_physFT_'+b+'_pow.npz'))['channels']
    freq_names = np.load(path.join(path_pow, su+'_'+phase+'_'+conds[0]+'_bipo_sel_physFT_'+b+'_pow.npz'))['fname']
    print (su, 'bad shape: ', bad_data.shape, 'good shape: ', good_data.shape)

    # =========================== Select Power for all elec 1 freq =================================                 
    #for elec_num in range(bad_data.shape[1]):
    for freq in freqs:
        #sel_elecs = [25,65,5,16,17,18,19,23,7,11,12,26,28,29,30,34]
        # 1 freq // Bad-Good conditions before (nfreq, nelec, nwin, ntrials)(after ntrials, nelecs, nwins)
        bad_data_elec = bad_data[freq,:,17:52,:].swapaxes(0,1).swapaxes(0,2)
        good_data_elec = good_data[freq,:,17:52,:].swapaxes(0,1).swapaxes(0,2)
        print ('data elec ', bad_data_elec.shape, good_data_elec.shape)
        nwin, freq_name = good_data.shape[1],freq_names[freq]
        print ('freq',freq_name)
# =============================  Classification Computation ============================================================           
        # create a data matrix, concatenate along the trial dimension
        x = np.concatenate((bad_data_elec, good_data_elec), axis=0)
        print ('Size of the concatenated data: ', x.shape, 'Number time windows : ', x.shape[2])
        #create label vector (0 for rest and 1 for odor)
        y = np.asarray([0]*bad_data_elec.shape[0] + [1]*good_data_elec.shape[0])
        print ('Size of label for classif: ', len(y))

        auc = np.array([])
        for t in range(x.shape[2]):
            X = x[:,:,t]
            #X = X.reshape(-1, 1)
            score_rep = []
            for i in range(10):
                skf = SKFold(n_splits=6, random_state=i)
                skf.get_n_splits(X, y)
                score_cv = []
                for train_index, test_index in skf.split(X, y):
                    clf = LDA()
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    clf.fit(X=X_train, y=y_train)
                    y_pred = clf.predict(X_test)
                    score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                score_rep.append(np.mean(score_cv))
            score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
            auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
        auc = np.swapaxes(auc,0,1)
        print(auc.shape)

        perm_scores = np.array([])
        for t in range(x.shape[2]):
            X = x[:,:,t]
            #X = X.reshape(-1, 1)
            perm_rep = []
            for perm in range(nperm):
                y_perm = y[permutation(len(y))]
                score_cv = []
                for train_index, test_index in skf.split(X, y_perm):
                    clf = LDA()
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y_perm[train_index], y_perm[test_index]
                    clf.fit(X=X_train, y=y_train)
                    y_pred = clf.predict(X_test)
                    score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                perm_rep.append(np.mean(score_cv))
            perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
            perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
            #print(perm_scores.shape)
        perm_scores = np.swapaxes(perm_scores,0,1)

        th_0_05_perm = perm_pvalue2level(perm_scores, p=0.05, maxst=True)
        th_0_01_perm = perm_pvalue2level(perm_scores, p=0.01, maxst=True)
        print('th_perm : ', th_0_05_perm[0], th_0_01_perm[0])

# ========================== Create a pvalue vector for uac measure ========================
        auc_pvals = []
        for i in range(auc.shape[1]):
            if np.mean(auc[:,i]) > th_0_01_perm[0]:
                auc_pvals.append(0.009)
            elif np.mean(auc[:,i]) > th_0_05_perm[0]:
                auc_pvals.append(0.04)
            else:
                auc_pvals.append(1)
        print (auc_pvals)

# ============================== PLOT POWER ANALYSIS + STATS & DECODING ACCURACY ===================================================
        
        #Filenames to save
        name_auc = (save_path+su +'_all_elecs_auc_Good_Bad_'+freq_name+'.npy')
        name_th_0_05_perm = (save_path+su +'_all_elecs_th_0_05_perm_'+freq_name+'.npy')
        name_th_0_01_perm = (save_path+su +'_all_elecs_th_0_01_perm_'+freq_name+'.npy')
        #plot_name = (save_path+str(freq)+'_'+freq_name+'/fig/'+su +'_Power_Good_Bad_'+str(elec)+'_'+str(elec_label)+'_('+str(elec_num)+').png')            
        #Save plots
        np.save(name_auc, auc)
        np.save(name_th_0_05_perm, th_0_05_perm[0])
        np.save(name_th_0_01_perm, th_0_01_perm[0])
#         plt.savefig(plot_name, dpi=300, bbox_inches='tight')
#         plt.clf()
#         plt.close() 
        del bad_data_elec, good_data_elec, X, auc
del bad_data, good_data

CHAF bad shape:  (7, 58, 64, 6) good shape:  (7, 58, 64, 15)
data elec  (6, 58, 35) (15, 58, 35)
freq theta
Size of the concatenated data:  (21, 58, 35) Number time windows :  35
Size of label for classif:  21


/home/karim/anaconda3/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(10, 35)
th_perm :  0.75 0.875
[0.04, 0.04, 1, 0.04, 0.04, 1, 1, 1, 1, 0.04, 1, 1, 1, 0.04, 0.04, 0.04, 0.009, 0.04, 0.04, 1, 1, 1, 1, 0.04, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
data elec  (6, 58, 35) (15, 58, 35)
freq alpha
Size of the concatenated data:  (21, 58, 35) Number time windows :  35
Size of label for classif:  21
(10, 35)
th_perm :  0.763888888889 0.888888888889
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 1, 1, 0.04, 0.04, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 0.04, 0.04, 0.04, 1, 1, 1, 1]
data elec  (6, 58, 35) (15, 58, 35)
freq beta
Size of the concatenated data:  (21, 58, 35) Number time windows :  35
Size of label for classif:  21
(10, 35)
th_perm :  0.763888888889 0.902777777778
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 0.04, 0.04, 0.04, 1, 0.04, 0.04, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 1, 0.04, 0.04]
data elec  (6, 58, 35) (15, 58, 35)
freq gamma1
Size of the concatenated data:  (21, 58, 35) Number time windows :  35
Size of label for classif:  21
(10, 35)
th_perm : 

In [6]:
path2save = path.join(save_path, 'Figures/')
phase, b ='odor', 'None'

for su in subjects:
    freq_names = np.load(path.join(path_pow, su+'_'+phase+'_low_bipo_sel_physFT_'+b+'_pow.npz'))['fname']

    # load all auc for all frequency bands
    #auc_f0 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[0]+'.npy')), axis=0)
    #auc_f1 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[1]+'.npy')), axis=0)
    auc_f2 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[2]+'.npy')), axis=0)
    auc_f3 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[3]+'.npy')), axis=0)
    auc_f4 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[4]+'.npy')), axis=0)
    auc_f5 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[5]+'.npy')), axis=0)
    auc_f6 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[6]+'.npy')), axis=0)
    #auc_f7 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[7]+'.npy')), axis=0)
    npts = auc_f2.shape[0]

    # Load threshold and find the max
    thr_05, thr_01 = np.array([]),np.array([])
    for i,f in enumerate(freq_names[2:7]):
        print(f)
        thr_f_05 = np.load(path.join(save_path,su+'_all_elecs_th_0_05_perm_'+f+'.npy'))
        thr_f_01 = np.load(path.join(save_path,su+'_all_elecs_th_0_01_perm_'+f+'.npy'))
        thr_05 = np.hstack((thr_05,thr_f_05)) if np.size(thr_05) else thr_f_05
        thr_01 = np.hstack((thr_01,thr_f_01)) if np.size(thr_01) else thr_f_01
    max_05, max_01 = np.max(thr_05), np.max(thr_01)
    print('max th values', max_05, max_01)

    # plot and figure parameters
    xfmt = ScalarFormatter(useMathText=True)
    xfmt.set_powerlimits((0,3))
    fig = plt.figure(1,figsize=(12,9))
    title = 'DA for '+su+' Low/High - All elecs - All freqs '
    fig.suptitle(title, fontsize=12)
    # Time vector to plot power
    step = 3500/npts
    times_plot = np.arange(-1000, 2500, step)
    print('time',times_plot.shape)

    # Plot the DA for all frequency bands
    #plt.plot(times_plot,auc_f0,'-',color='m',label=freq_names[0],linewidth=1.5)
    #plt.plot(times_plot,auc_f1,'-',color='b',label=freq_names[1],linewidth=1.5)
    plt.plot(times_plot,auc_f2,'-',color='b',label=freq_names[2],linewidth=1.5)
    plt.plot(times_plot,auc_f3,'-',color='g',label=freq_names[3],linewidth=1.5)
    plt.plot(times_plot,auc_f4,'-',color='y',label=freq_names[4],linewidth=1.5)
    plt.plot(times_plot,auc_f5,'-',color='orange',label=freq_names[5],linewidth=1.5)
    plt.plot(times_plot,auc_f6,'-',color='r',label=freq_names[6],linewidth=1.5)
    #plt.plot(times_plot,auc_f7,'-',color='orange',label=freq_names[7],linewidth=1.5)
    addLines(plt.gca(), vLines=[0], vColor=['#000000']*2, vWidth=[1]*2, hLines=[0.5], 
         hColor=['#000000'], hWidth=[1])
    rmaxis(plt.gca(), ['right','top'])
    plt.legend(loc=0, handletextpad=0.1, frameon=False)
    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
    plt.plot(times_plot, max_05*np.ones(len(times_plot)), '--', color='black', linewidth=1)
    plt.plot(times_plot, max_01*np.ones(len(times_plot)), '--', color='orange', linewidth=1)
    #plt.show()
    plot_name = (path2save+su +'_DA_Good_Bad_MF_Elec_by_freq.png')            
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')
    plt.clf()
    plt.close()

theta
alpha
beta
gamma1
gamma2
max th values 0.791666666667 0.916666666667
time (35,)
theta
alpha
beta
gamma1
gamma2
max th values 0.75 0.916666666667
time (35,)
theta
alpha
beta
gamma1
gamma2
max th values 0.722222222222 0.819444444444
time (35,)
theta
alpha
beta
gamma1
gamma2
max th values 0.708333333333 0.805555555556
time (35,)
theta
alpha
beta
gamma1
gamma2
max th values 0.634259259259 0.729166666667
time (35,)
theta
alpha
beta
gamma1
gamma2
max th values 0.75 0.847222222222
time (35,)
theta
alpha
beta
gamma1
gamma2
max th values 0.644444444444 0.75
time (35,)


### Plot MF across time by freq by patient

In [ ]:
from os import path
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from brainpipe.visual import *

# PATH TO DATA
st = study('Olfacto')
path_pow = path.join(st.path, 'feature/0_Power_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/0_Classif_Power_Encoding_EpiPerf_LowHigh_None/MF_elecs/')
su, phase, b = 'LEFC', 'odor', 'None'
freq_names = np.load(path.join(path_pow, su+'_'+phase+'_low_bipo_sel_physFT_'+b+'_power.npz'))['fname']

# Load threshold and find the max
thr_05, thr_01 = np.array([]),np.array([])
for f in freq_names:
    thr_f_05 = np.load(path.join(save_path,su+'_sel_elecs_th_0_05_perm_'+f+'.npy'))
    thr_f_01 = np.load(path.join(save_path,su+'_sel_elecs_th_0_01_perm_'+f+'.npy'))
    thr_05 = np.hstack((thr_05,thr_f_05)) if np.size(thr_05) else thr_f_05
    thr_01 = np.hstack((thr_01,thr_f_01)) if np.size(thr_01) else thr_f_01
max_05, max_01 = np.max(thr_05), np.max(thr_01)
print('max th values', max_05, max_01)

# load all auc for all frequency bands
#auc_f0 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[0]+'.npy')), axis=0)
#auc_f1 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[1]+'.npy')), axis=0)
auc_f2 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[2]+'.npy')), axis=0)
auc_f3 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[3]+'.npy')), axis=0)
auc_f4 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[4]+'.npy')), axis=0)
auc_f5 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[5]+'.npy')), axis=0)
auc_f6 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[6]+'.npy')), axis=0)
auc_f7 = np.mean(np.load(path.join(save_path,su+'_sel_elecs_auc_Good_Bad_'+freq_names[7]+'.npy')), axis=0)

#Generate the 3 values for each frequency bands
#auc_f0_wins = [np.mean(auc_f0[0:12]),np.mean(auc_f0[12:36]),np.mean(auc_f0[36:49])]
#auc_f1_wins = [np.mean(auc_f1[0:12]),np.mean(auc_f1[12:36]),np.mean(auc_f1[36:49])]
auc_f2_wins = [np.mean(auc_f2[0:12]),np.mean(auc_f2[12:36]),np.mean(auc_f2[36:49])]
auc_f3_wins = [np.mean(auc_f3[0:12]),np.mean(auc_f3[12:36]),np.mean(auc_f3[36:49])]
auc_f4_wins = [np.mean(auc_f4[0:12]),np.mean(auc_f4[12:36]),np.mean(auc_f4[36:49])]
auc_f5_wins = [np.mean(auc_f5[0:12]),np.mean(auc_f5[12:36]),np.mean(auc_f5[36:49])]
auc_f6_wins = [np.mean(auc_f6[0:12]),np.mean(auc_f6[12:36]),np.mean(auc_f6[36:49])]
auc_f7_wins = [np.mean(auc_f7[0:12]),np.mean(auc_f7[12:36]),np.mean(auc_f7[36:49])]

# Prepare the plots
ind = np.arange(3)
print(ind)# the locations for the 3 phases
width = 0.12       # the width of the bars

fig = plt.figure(1,figsize=(10,5))
ax = plt.subplot()
#rects1 = ax.bar(ind, auc_f0_wins, width, color='m')
#rects2 = ax.bar(ind + width, auc_f1_wins, width, color='b')
rects3 = ax.bar(ind + width*2, auc_f2_wins, width, color='c')
rects4 = ax.bar(ind + width*3, auc_f3_wins, width, color='g')
rects5 = ax.bar(ind + width*4, auc_f4_wins, width, color='y')
rects6 = ax.bar(ind + width*5, auc_f5_wins, width, color='orange')
rects7 = ax.bar(ind + width*6, auc_f6_wins, width, color='r')
#rects8 = ax.bar(ind + width*7, auc_f7_wins, width, color='orange')

# add some text for labels, title and axes ticks
ax.set_ylabel('AUC Scores')
ax.set_title('Scores by frequency and phase')
ax.set_xticks(ind + width*6 / 2)
ax.set_xticklabels(('pre', 'odor', 'post'))
ax.set_ylim(bottom=0., top=1.)

ax.legend((rects1[0], rects2[0], rects3[0], rects4[0], rects5[0], rects6[0], rects7[0]), 
    ('VLFC', 'delta', 'theta','alpha','beta','gamma1', 'gamma2'), loc=0)

addLines(plt.gca(), hLines=[max_05,max_01], hColor=['grey']*2, hWidth=[2]*2, hShape=['--', '--'])
#plt.show()
plot_name = (save_path+su +'_Mean_AUC_Good_Bad_MF_Elec_by_freq.png')            
plt.savefig(plot_name, dpi=300, bbox_inches='tight')
plt.clf()
plt.close()

### Plot Max MF before/after Inspi by freq by patient

In [19]:
from os import path
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from brainpipe.visual import *

# PATH TO DATA
st = study('Olfacto')
path_pow = path.join(st.path, 'feature/0_Power_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/0_Classif_Power_Encoding_EpiPerf_LowHigh_None/MF_elecs/')
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ']
phase, b = 'odor', 'None'
freq_names = np.load(path.join(path_pow, su+'_'+phase+'_low_bipo_sel_physFT_'+b+'_pow.npz'))['fname']

# Load threshold and find the max

for su in subjects:
    thr_05, thr_01 = np.array([]),np.array([])
    for i,f in enumerate(freq_names[2:6]):
        thr_f_05 = np.load(path.join(save_path,su+'_all_elecs_th_0_05_perm_'+f+'.npy'))
        thr_f_01 = np.load(path.join(save_path,su+'_all_elecs_th_0_01_perm_'+f+'.npy'))
        thr_05 = np.hstack((thr_05,thr_f_05)) if np.size(thr_05) else thr_f_05
        thr_01 = np.hstack((thr_01,thr_f_01)) if np.size(thr_01) else thr_f_01
    max_05, max_01 = np.max(thr_05), np.max(thr_01)
    print('max th values', max_05, max_01)

    # load all auc for all frequency bands
    #auc_f0 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[0]+'.npy')), axis=0)
    #auc_f1 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[1]+'.npy')), axis=0)
    auc_f2 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[2]+'.npy')), axis=0)
    auc_f3 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[3]+'.npy')), axis=0)
    auc_f4 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[4]+'.npy')), axis=0)
    auc_f5 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[5]+'.npy')), axis=0)
    auc_f6 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[6]+'.npy')), axis=0)

    #Generate the 3 values for each frequency bands
    #auc_f0_wins = [np.max(auc_f0[0:9]),np.max(auc_f0[9:36]),np.max(auc_f0[36:49])]
    #auc_f1_wins = [np.max(auc_f1[0:9]),np.max(auc_f1[9:36]),np.max(auc_f1[36:49])]
    auc_f2_wins = [np.max(auc_f2[0:9]),np.max(auc_f2[9:35])]
    auc_f3_wins = [np.max(auc_f3[0:9]),np.max(auc_f3[9:35])]
    auc_f4_wins = [np.max(auc_f4[0:9]),np.max(auc_f4[9:35])]
    auc_f5_wins = [np.max(auc_f5[0:9]),np.max(auc_f5[9:35])]
    auc_f6_wins = [np.max(auc_f6[0:9]),np.max(auc_f6[9:35])]

    # Prepare the plots
    ind = np.arange(2)
    print(ind)# the locations for the 3 phases
    width = 0.15       # the width of the bars

    fig = plt.figure(1,figsize=(7,5))
    ax = plt.subplot()
    #rects1 = ax.bar(ind, auc_f0_wins, width, color='m')
    #rects2 = ax.bar(ind + width, auc_f1_wins, width, color='b')
    rects3 = ax.bar(ind + width, auc_f2_wins, width, color='c')
    rects4 = ax.bar(ind + width*2, auc_f3_wins, width, color='g')
    rects5 = ax.bar(ind + width*3, auc_f4_wins, width, color='y')
    rects6 = ax.bar(ind + width*4, auc_f5_wins, width, color='orange')
    rects7 = ax.bar(ind + width*5, auc_f6_wins, width, color='r')
    #rects8 = ax.bar(ind + width*7, auc_f7_wins, width, color='orange')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('AUC Scores')
    ax.set_title(su + ' Scores by frequency and phase')
    ax.set_xticks(ind + width*5/2)
    ax.set_xticklabels(('pre', 'odor'))
    ax.set_ylim(bottom=0.3, top=1.1)

    ax.legend((rects3[0], rects4[0], rects5[0], rects6[0], rects7[0]), 
        ('theta','alpha','beta','gamma1', 'gamma2'), loc=0)

    addLines(plt.gca(), hLines=[max_05,max_01], hColor=['grey']*2, hWidth=[2]*2, hShape=['--', '--'])
    #plt.show()
    plot_name = (path2save+su +'_Max_AUC_Good_Bad_MF_Elec_by_freq.png')            
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')
    plt.clf()
    plt.close()

-> Olfacto loaded
max th values 0.644444444444 0.75
[0 1]
max th values 0.791666666667 0.916666666667
[0 1]
max th values 0.75 0.916666666667
[0 1]
max th values 0.722222222222 0.819444444444
[0 1]
max th values 0.708333333333 0.805555555556
[0 1]
max th values 0.631944444444 0.729166666667
[0 1]
max th values 0.75 0.847222222222
[0 1]


### Plot Max MF before/after inspi for all patients by freq

In [28]:
from os import path
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from brainpipe.visual import *

# PATH TO DATA
st = study('Olfacto')
path_pow = path.join(st.path, 'feature/0_Power_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/0_Classif_Power_Encoding_EpiPerf_LowHigh_None/MF_elecs/')
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ']
phase, b = 'odor', 'None'
freq_names = np.load(path.join(path_pow, su+'_'+phase+'_low_bipo_sel_physFT_'+b+'_pow.npz'))['fname']

# Load threshold Max MF perm all freqs all patients
thr_05, thr_01 = np.array([]),np.array([])
for su in subjects:
    for i,f in enumerate(freq_names[2:6]):
        thr_f_05 = np.load(path.join(save_path,su+'_all_elecs_th_0_05_perm_'+f+'.npy'))
        thr_f_01 = np.load(path.join(save_path,su+'_all_elecs_th_0_01_perm_'+f+'.npy'))
        thr_05 = np.hstack((thr_05,thr_f_05)) if np.size(thr_05) else thr_f_05
        thr_01 = np.hstack((thr_01,thr_f_01)) if np.size(thr_01) else thr_f_01
max_05, max_01 = np.max(thr_05), np.max(thr_01)
print('max th values', max_05, max_01)

# Load all MF by freq and find max MF
max_0, max_1, max_2, max_3, max_4 = np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
for su in subjects:
    auc_f2 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[2]+'.npy')), axis=0)
    auc_f3 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[3]+'.npy')), axis=0)
    auc_f4 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[4]+'.npy')), axis=0)
    auc_f5 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[5]+'.npy')), axis=0)
    auc_f6 = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+freq_names[6]+'.npy')), axis=0)

    auc_f2_wins = [np.max(auc_f2[0:9]),np.max(auc_f2[9:35])]
    auc_f3_wins = [np.max(auc_f3[0:9]),np.max(auc_f3[9:35])]
    auc_f4_wins = [np.max(auc_f4[0:9]),np.max(auc_f4[9:35])]
    auc_f5_wins = [np.max(auc_f5[0:9]),np.max(auc_f5[9:35])]
    auc_f6_wins = [np.max(auc_f6[0:9]),np.max(auc_f6[9:35])]
    
    max_0 = np.vstack((max_0, auc_f2_wins)) if np.size(max_0) else auc_f2_wins
    max_1 = np.vstack((max_1, auc_f3_wins)) if np.size(max_1) else auc_f3_wins
    max_2 = np.vstack((max_2, auc_f4_wins)) if np.size(max_2) else auc_f4_wins
    max_3 = np.vstack((max_3, auc_f5_wins)) if np.size(max_3) else auc_f5_wins
    max_4 = np.vstack((max_4, auc_f6_wins)) if np.size(max_4) else auc_f6_wins
print('max freqs',max_0.shape, max_1.shape, max_2.shape, max_3.shape, max_4.shape)
max_mean0,max_mean1,max_mean2 = np.max(max_0, axis=0),np.max(max_1, axis=0),np.max(max_2, axis=0)
max_mean3,max_mean4 = np.max(max_3, axis=0),np.max(max_4, axis=0)
#max_err0,max_err1,max_err2 = np.std(max_0, axis=0),np.std(max_1, axis=0),np.std(max_2, axis=0)
#max_err3,max_err4 = np.std(max_3, axis=0),np.std(max_4, axis=0)
max_err0,max_err1,max_err2 = 0,0,0
max_err3,max_err4 = 0,0
print('max freqs',max_0.shape, max_1.shape, max_2.shape, max_3.shape, max_4.shape)

# Prepare the plots
ind = np.arange(2)
print(ind)# the locations for the 3 phases
width = 0.15       # the width of the bars

fig = plt.figure(1,figsize=(7,5))
ax = plt.subplot()
#rects1 = ax.bar(ind, auc_f0_wins, width, color='m')
#rects2 = ax.bar(ind + width, auc_f1_wins, width, color='b')
rects3 = ax.bar(ind + width, max_mean0, width, color='c', yerr=max_err0)
rects4 = ax.bar(ind + width*2, max_mean1, width, color='g', yerr=max_err1)
rects5 = ax.bar(ind + width*3, max_mean2, width, color='y', yerr=max_err2)
rects6 = ax.bar(ind + width*4, max_mean3, width, color='orange', yerr=max_err3)
rects7 = ax.bar(ind + width*5, max_mean4, width, color='r', yerr=max_err4)
#rects8 = ax.bar(ind + width*7, auc_f7_wins, width, color='orange')

# add some text for labels, title and axes ticks
ax.set_ylabel('AUC Scores')
ax.set_title(' Scores by frequency and phase')
ax.set_xticks(ind + width*5/2)
ax.set_xticklabels(('pre', 'odor'))
ax.set_ylim(bottom=0.3, top=1.1)

ax.legend((rects3[0], rects4[0], rects5[0], rects6[0], rects7[0]), 
    ('theta','alpha','beta','gamma1', 'gamma2'), loc=0)

addLines(plt.gca(), hLines=[max_05,max_01], hColor=['grey']*2, hWidth=[2]*2, hShape=['--', '--'])
#plt.show()
plot_name = (path2save+'All_patients_Max_Max_AUC_Good_Bad_MF_Elec_by_freq.png')
plot_name2 = (path2save+'All_patients_Max_Max_AUC_Good_Bad_MF_Elec_by_freq.pdf') 
plt.savefig(plot_name, dpi=300, bbox_inches='tight')
plt.savefig(plot_name2, dpi=300, bbox_inches='tight')
plt.clf()
plt.close()

-> Olfacto loaded
max th values 0.791666666667 0.916666666667
max freqs (7, 2) (7, 2) (7, 2) (7, 2) (7, 2)
max freqs (7, 2) (7, 2) (7, 2) (7, 2) (7, 2)
[0 1]


### Plot by freq and phase for all subjects

In [ ]:
from os import path
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from brainpipe.visual import *

# PATH TO DATA
st = study('Olfacto')
path_pow = path.join(st.path, 'feature/0_Power_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/0_Classif_Power_Encoding_EpiPerf_LowHigh_None/MF_elecs/')
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ']
phase, b = 'odor', 'None'
freq_names = ['theta', 'alpha', 'beta', 'gamma1', 'gamma2']

# Load threshold and find the max
for f in freq_names:
    auc_MF,thr_05, thr_01 = np.array([]),np.array([]),np.array([])
    for su in subjects:
        auc = np.mean(np.load(path.join(save_path,su+'_all_elecs_auc_Good_Bad_'+f+'.npy')), axis=0)
        auc_wins = [np.max(auc[0:9]),np.max(auc[9:36]),np.max(auc[36:49])]
        thr_f_05 = np.load(path.join(save_path,su+'_all_elecs_th_0_05_perm_'+f+'.npy'))
        thr_f_01 = np.load(path.join(save_path,su+'_all_elecs_th_0_01_perm_'+f+'.npy'))
        auc_MF = np.vstack((auc_MF,auc_wins)) if np.size(auc_MF) else auc_wins
        thr_05 = np.hstack((thr_05,thr_f_05)) if np.size(thr_05) else thr_f_05
        thr_01 = np.hstack((thr_01,thr_f_01)) if np.size(thr_01) else thr_f_01
    max_05, max_01 = np.max(thr_05), np.max(thr_01) #max for all subjects
    print('max th values', max_05, max_01)
    print('auc val',auc_MF.shape)
    
    # Prepare the plots
    ind = np.arange(3)
    print(ind)# the locations for the 3 phases
    width = 0.12       # the width of the bars

    fig = plt.figure(1,figsize=(10,5))
    ax = plt.subplot()
    rects1 = ax.bar(ind, auc_MF[0], width, color='m')
    rects2 = ax.bar(ind + width, auc_MF[1], width, color='b')
    rects3 = ax.bar(ind + width*2, auc_MF[2], width, color='c')
    rects4 = ax.bar(ind + width*3, auc_MF[3], width, color='g')
    rects5 = ax.bar(ind + width*4, auc_MF[4], width, color='y')
    rects6 = ax.bar(ind + width*5, auc_MF[5], width, color='orange')
    rects7 = ax.bar(ind + width*6, auc_MF[6], width, color='r')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('AUC Scores')
    ax.set_title(' Scores for '+f+' by subject and phase')
    ax.set_xticks(ind + width*6 / 2)
    ax.set_xticklabels(('pre', 'odor', 'post'))
    ax.set_ylim(bottom=0., top=1.)

    ax.legend((rects1[0], rects2[0], rects3[0], rects4[0], rects5[0], rects6[0], rects7[0]), 
        ('LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ'), loc=0)

    addLines(plt.gca(), hLines=[max_05,max_01], hColor=['grey']*2, hWidth=[2]*2, hShape=['--', '--'])
    #plt.show()
    plot_name = (save_path+'Max_AUC_Good_Bad_MF_'+f+'.png')            
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')
    plt.clf()
    plt.close()